In [5]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from PIL import Image
import pytorch_lightning as pl


In [6]:
class BrainDataModule:
    def __init__(self, data_dir, batch_size, num_classes):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform_train = transforms.Compose([
            transforms.Resize((256, 256)),  # Giữ nguyên kích thước
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomAffine(5, translate=[0, .1], scale=[.9, 1.1]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Giữ nguyên kích thước
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.num_classes = num_classes  # Thêm thuộc tính num_classes

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train = ImageFolder(
                os.path.join(self.data_dir, "train"), transform=self.transform_train
            )
            self.val = ImageFolder(
                os.path.join(self.data_dir, "val"), transform=self.transform
            )

        if stage == 'test' or stage is None:
            self.test = ImageFolder(
                os.path.join(self.data_dir, "test"), transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)


In [50]:
# Đường dẫn đến dữ liệu của bạn
data_dir = "G:/BrainTumorV2/Dataset(1)"

# Số lớp của mô hình Random Forest
num_classes_rf = 4  # Thay bằng số lớp thích hợp

# Tạo data module
data_module = BrainDataModule(data_dir, batch_size=32, num_classes=num_classes_rf)
data_module.setup()

# Load dữ liệu
train_dataloader = data_module.train_dataloader()
val_dataloader = data_module.val_dataloader()
# Tạo một PyTorch Lightning Trainer với GPU (nếu có)
trainer = pl.Trainer(max_epochs=50, accelerator='gpu')
# Chuẩn bị mô hình Random Forest
input_size = 256 * 256 * 3  # Kích thước ảnh 256x256 và có 3 channels
rf_model = RandomForestClassifier(n_estimators=2002,
    max_depth=50,
    min_samples_split=22,
    min_samples_leaf=15,
    random_state=46)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [51]:
# Huấn luyện mô hình
all_labels = []
all_predictions = []

for batch in tqdm(train_dataloader):
    x, y = batch
    x = x.view(x.size(0), -1)  # Flatten ảnh thành vector
    rf_model.fit(x.numpy(), y.numpy())
    all_labels.extend(y.numpy())
    all_predictions.extend(rf_model.predict(x.numpy()))

# Đánh giá mô hình trên tập validation
val_labels = []
val_predictions = []

for batch in tqdm(val_dataloader):
    x_val, y_val = batch
    x_val = x_val.view(x_val.size(0), -1)
    val_labels.extend(y_val.numpy())
    val_predictions.extend(rf_model.predict(x_val.numpy()))

# In độ chính xác và báo cáo phân loại
train_accuracy = accuracy_score(all_labels, all_predictions)
val_accuracy = accuracy_score(val_labels, val_predictions)

print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Accuracy: {val_accuracy}")

val_classification_report = classification_report(val_labels, val_predictions)
print("Validation Classification Report:\n", val_classification_report)


100%|██████████| 10/10 [00:05<00:00,  1.89it/s]

Training Accuracy: 0.3745958429561201
Validation Accuracy: 0.29449838187702265
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        90
           1       0.29      1.00      0.46        91
           2       0.00      0.00      0.00        44
           3       0.00      0.00      0.00        84

    accuracy                           0.29       309
   macro avg       0.07      0.25      0.11       309
weighted avg       0.09      0.29      0.13       309




c:\Users\nguye\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nguye\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nguye\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
